# **연관규칙분석을 통한 웹 페이지 구조 변경**  
---
> **목차(Context)**

* 🥉Session 1 - 문제 및 목적 정의
* 🥈Session 2 - 시퀀스 데이터 연관규칙 탐색
* 🥇Session 3 - 해결 방안

## **🥉Session 1 - 문제 및 목적 정의**  
---

> **문제 정의** 

```
Web을 방문하는 고객들의 체류 시간을 늘리기 위한 페이지 구성을 알아본다.

```

> **목적 정의** 

```
Web을 방문하는 고객들의 체류 시간을 늘리기 위함이다.
고객의 체류 시간을 높이면, 다음의 효과를 기대할 수 있다.
1. 우리 제품이 Web을 방문하는 고객들에게 자연스럽게 오래 노출된다.
2. SEO 점수가 높아져서 검색 결과 상단에 노출될 수 있다.
```

In [1]:
import os
import pandas as pd

os.chdir('/content/drive/MyDrive/Association Rule Analysis')

In [2]:
df = pd.read_csv(
    "페이지내_사용자_이동.csv", 
    engine="python", 
    encoding='cp949')

df.head()

,고객ID,방문 페이지,순서
0,0,페이지C,1
1,0,페이지E,2
2,0,페이지B,3
3,0,페이지F,4
4,0,페이지C,5


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   고객ID    732 non-null    int64 
 1   방문 페이지  732 non-null    object
 2   순서      732 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 17.3+ KB


In [4]:
# 결측 여부 확인
df.isnull().sum()

고객ID      0
방문 페이지    0
순서        0
dtype: int64

## **🥈Session 2 - 시퀀스 데이터 연관규칙 탐색**  
---
연관 규칙 탐색을 위해 **Apriori 원리**를 사용한다. 연관 규칙의 평가를 위한 2가지 기준이 있다. 지지도(Support), 신뢰도(Confidence) 이다.

* 지지도. 해당 아이템 집합이 전체 트랜젝션 데이터에서 발생한 비율을 의미한다. 
* 신뢰도. 부모 아이템 집합이 등장한 트랜젝션 데이터에서 자식 아이템 집합이 발생한 비율을 의미한다.

시퀀스 데이터로부터 발생 가능한 모든 조합을 탐색하는 것은 경우의 수가 많아서 현실적으로 불가능하다. 그래서 효율적 탐색을 위해 지지도, 신뢰도에 대한 Apriori 원리를 활용한다.

### Apriori 원리를 활용한 연관규칙 탐색
* 지지도에 대한 Apriori 원리를 활용하여 최대 빈발 아이템 집합을 찾는다.
* 신뢰도에 대한 Apriori 원리를 활용하여 최대 빈발 아이템 집합으로부터 최소 신뢰도 이하의 아이템 집합을 탐색하는데 활용한다.

In [5]:
# 기존 데이터를 Sequence Data로 변환한다.
# Sequence 데이터의 순서가 섞이지 않게끔 DataFrame 내부의 순서를 기준으로 정렬한다.
df['순서'] = df['순서'].astype(int)
df = df.sort_values(by=['고객ID', '순서'])
df.head()

,고객ID,방문 페이지,순서
0,0,페이지C,1
1,0,페이지E,2
2,0,페이지B,3
3,0,페이지F,4
4,0,페이지C,5


In [6]:
import numpy as np

# 고객 ID마다 방문한 순서대로 np.array에 담는다.
page_sequence_per_order = df.groupby('고객ID')['방문 페이지'].apply(np.array)
page_sequence_per_order.head()

고객ID
0    [페이지C, 페이지E, 페이지B, 페이지F, 페이지C, 페이지D, 페이지J, 메인,...
1    [페이지B, 메인, 메인, 페이지A, 페이지F, 페이지C, 페이지I, 페이지E, 페...
2    [페이지F, 페이지J, 페이지D, 페이지G, 페이지C, 메인, 페이지I, 페이지J,...
3    [페이지I, 페이지I, 페이지J, 페이지J, 페이지C, 페이지A, 페이지H, 페이지...
4    [페이지J, 페이지H, 페이지G, 페이지G, 페이지E, 페이지C, 페이지A, 페이지...
Name: 방문 페이지, dtype: object

In [ ]:
# 신뢰도, 지지도에 대한 Apriori 원리를 바탕으로 
# 최대 빈발 아이템 집합, 그리고 최소 신뢰도 이하 집합을 찾기 위한 함수를 선언한다.

In [7]:
# 순서를 고려한 최대 빈발 아이템 집합을 찾기 위해 개별 페이지 집합을 선언한다.
page_set = df['방문 페이지'].unique()
page_set

array(['페이지C', '페이지E', '페이지B', '페이지F', '페이지D', '페이지J', '메인', '페이지G',
       '페이지A', '페이지I', '페이지H'], dtype=object)

In [8]:
from itertools import product

# record에 pattern이 거리 L 내에 존재하는지를 판단한다.
def contain_pattern(record, pattern, L):
    output = False    
    if set(record) & set(pattern) != set(pattern):
        return False    
    else:
        # 개별 item의 index를 담은 list를 반환한다.
        pattern_index_list = [np.where(record == item)[0] for item in pattern]
        
        # 개별 item끼리의 거리가 0보다 크고, L 보다 작은 지 확인한다.
        # 모두 L보다 작은 경우에만, True를 반환한다.
        for pattern_index in product(*pattern_index_list): 
            distance = np.array(pattern_index)[1:] - np.array(pattern_index)[:-1]
            if sum((distance <= L) & (distance > 0)) == (len(pattern_index) - 1):
                output = True
                break
        
        return output


# 지지도에 대한 Apriori 원리를 이용하여 최대 빈발 아이템 집합을 찾는다.
def find_maximum_frequent_sequence_item(
    item_set, 
    sequence_data, 
    min_support=0.01, 
    L=1):

    queue = []
    maximum_frequent_sequence_item = []
    len_seq_data = len(sequence_data)
    
    # 개별 item의 빈발 여부 판단
    for item in item_set:
        occurence = sequence_data.apply(
            contain_pattern, pattern=[item], L=L).sum()
        
        if occurence / len_seq_data >= min_support:
            queue.append([item])

    while queue:
        current_pattern = queue.pop()
        check_maximum_frequent = True
        len_current_pattern = len(current_pattern)

        for item in item_set:
            occurence = sequence_data.apply(
                contain_pattern, pattern=current_pattern + [item], L=L).sum()
            if occurence / len_seq_data >= min_support:
                check_maximum_frequent = False 
                queue.append(current_pattern + [item])
        
        # 최대 빈발 아이템 집합인 경우
        # 현재 아이템 집합에서 모든 모집합이 빈발하지 않고, 길이가 2 이상인 경우만 최대 빈발 아이템 집합이 된다.
        if check_maximum_frequent and len_current_pattern > 1:
            maximum_frequent_sequence_item.append(current_pattern)        
    
    return maximum_frequent_sequence_item

def generate_association_rules(
    maximum_frequent_sequence_item, 
    sequence_data, 
    min_support=0.01, 
    min_confidence=0.01, 
    L = 1):
    
    len_seq_data = len(sequence_data)
    result = {"부모":[], "자식":[], "지지도":[], "신뢰도":[]}
    
    # 최대 빈발 아이템 집합으로부터 만들어지는 연관 규칙의 신뢰도를 파악한다.
    for sequence_item in maximum_frequent_sequence_item:
        co_occurence = sequence_data.apply(
            contain_pattern, pattern=sequence_item, L=L).sum()
        support = co_occurence / len_seq_data
        
        if co_occurence > min_support:
            # 신뢰도에 대한 Apriori의 원리를 적용하기 위해 부모가 더 큰 경우부터 시작한다.
            for i in range(len(sequence_item)-1, 0, -1):
                antecedent = sequence_item[:i]
                consequent = sequence_item[i:]
                antecedent_occurence = sequence_data.apply(
                    contain_pattern, pattern=antecedent, L=L).sum()
                
                confidence = co_occurence / antecedent_occurence
                
                # 신뢰도에 대한 Apriori 원리에 따라 최소 신뢰도를 이하라면, 부분 집합에 대한 추가 탐색은 진행하지 않는다.
                # 그래서 break를 통해 for문을 탈출한다.
                # 최소 신뢰도를 만족한다면, 좋은 연관 규칙이라 판단하여 추가한다.

                # 최소 신뢰도 이상인 경우
                if confidence > min_confidence:
                    result['부모'].append(antecedent)
                    result['자식'].append(consequent)
                    result['지지도'].append(support)
                    result['신뢰도'].append(confidence)
                
                # 최소 신뢰도 이하인 경우
                # confidence <= min_confidence
                else: 
                    break
            
    return pd.DataFrame(result)

In [9]:
maximum_frequent_sequence_item = find_maximum_frequent_sequence_item(
    page_set, page_sequence_per_order, min_support = 0.02, L = 1)

result = generate_association_rules(
    maximum_frequent_sequence_item, 
    page_sequence_per_order, 
    min_support=0.01, 
    min_confidence=0.01, 
    L=1)

result.head()

,부모,자식,지지도,신뢰도
0,[페이지H],[페이지H],0.05,0.121951
1,[페이지H],[페이지I],0.03,0.073171
2,"[페이지H, 페이지A]",[페이지D],0.02,0.400000
3,[페이지H],"[페이지A, 페이지D]",0.02,0.048780
4,[페이지H],[메인],0.03,0.073171


In [10]:
result = result.sort_values(by=['지지도', '신뢰도'], ascending=False)
result = result.reset_index(drop=True)
result.head()

,부모,자식,지지도,신뢰도
0,[페이지B],[페이지J],0.07,0.155556
1,[페이지A],[페이지C],0.07,0.148936
2,[페이지J],[페이지D],0.07,0.142857
3,[페이지J],[페이지C],0.07,0.142857
4,[메인],[페이지H],0.06,0.150000


In [11]:
cp_result = result.copy()

for col in ['부모', '자식']:
    cp_result[col] = cp_result[col].apply(lambda x : ' '.join(x))

cp_result = cp_result.sort_values(
    by=['지지도', '신뢰도'], ascending=False).reset_index(drop=True)

cp_result.head()

,부모,자식,지지도,신뢰도
0,페이지B,페이지J,0.07,0.155556
1,페이지A,페이지C,0.07,0.148936
2,페이지J,페이지D,0.07,0.142857
3,페이지J,페이지C,0.07,0.142857
4,메인,페이지H,0.06,0.150000


In [12]:
cp_result.head(10)

,부모,자식,지지도,신뢰도
0,페이지B,페이지J,0.07,0.155556
1,페이지A,페이지C,0.07,0.148936
2,페이지J,페이지D,0.07,0.142857
3,페이지J,페이지C,0.07,0.142857
4,메인,페이지H,0.06,0.150000
5,메인,페이지C,0.06,0.150000
6,페이지H,페이지J,0.06,0.146341
7,페이지I,페이지A,0.06,0.139535
8,페이지F,페이지H,0.06,0.139535
9,페이지G,페이지G,0.06,0.136364


In [13]:
# 페이지 C와 연관성이 높은 페이지는 다음과 같다.
# 연관규칙탐색의 결과를 토대로 기존 페이지의 구성을 변화한다.
cp_result[cp_result['부모'] == '페이지C']

,부모,자식,지지도,신뢰도
21,페이지C,페이지F,0.05,0.092593
35,페이지C,페이지B,0.04,0.074074
57,페이지C,페이지H,0.03,0.055556
58,페이지C,페이지D,0.03,0.055556
208,페이지C,페이지I 페이지B,0.02,0.037037
209,페이지C,페이지A 페이지H 페이지B,0.02,0.037037
210,페이지C,페이지G,0.02,0.037037
211,페이지C,메인 페이지A 페이지J,0.02,0.037037
212,페이지C,메인 페이지E,0.02,0.037037
213,페이지C,페이지J 페이지I,0.02,0.037037


In [14]:
parents_list = cp_result['부모'].unique()
correlation_dict = {}

# 개별 페이지에서 지지도, 신뢰도 Top3를 기준으로 json 파일을 생성한다.
for parent in parents_list:
    temp_df = cp_result[cp_result['부모'] == parent]
    correlation_dict[parent] = list(temp_df['자식'].values[:3])

correlation_dict

{'페이지B': ['페이지J', '페이지I', '페이지G'],
 '페이지A': ['페이지C', '페이지E', '페이지I'],
 '페이지J': ['페이지D', '페이지C', '페이지E'],
 '메인': ['페이지H', '페이지C', '페이지B'],
 '페이지H': ['페이지J', '페이지H', '페이지E'],
 '페이지I': ['페이지A', '페이지J', '페이지F'],
 '페이지F': ['페이지H', '페이지D', '페이지A'],
 '페이지G': ['페이지G', '페이지D', '페이지E'],
 '페이지D': ['페이지E', '페이지D', '페이지C'],
 '페이지E': ['페이지I', '페이지J', '페이지D'],
 '페이지C': ['페이지F', '페이지B', '페이지H'],
 '페이지B 페이지F': ['페이지J'],
 '페이지B 페이지D': ['페이지B'],
 '페이지I 메인': ['페이지B'],
 '페이지G 페이지B': ['페이지F'],
 '페이지F 페이지J': ['페이지D'],
 '페이지J 페이지J': ['페이지C', '페이지D', '페이지F'],
 '페이지A 페이지G 페이지J': ['페이지E'],
 '페이지J 페이지I 페이지D': ['페이지F'],
 '페이지J 메인 페이지J': ['페이지F'],
 '페이지D 페이지J 페이지J': ['페이지F'],
 '페이지B 페이지E 페이지F': ['페이지G'],
 '페이지C 페이지A 페이지H': ['페이지B'],
 '페이지C 메인 페이지A': ['페이지J'],
 '페이지I 페이지H': ['페이지E'],
 '페이지A 페이지F': ['페이지C'],
 '페이지D 페이지J': ['페이지J 페이지F'],
 '페이지F 페이지B': ['페이지J'],
 '페이지I 페이지D': ['페이지F'],
 '페이지I 페이지E': ['페이지E'],
 '페이지A 페이지H': ['페이지B'],
 '페이지G 페이지J': ['페이지E'],
 '페이지J 페이지I': ['페이지D 페이지F'],
 '페이지D 페이지A': ['페이지E'],
 '페이지B 페이지

## **🥇Session 3 - 해결 방안**  
---

* 연관 규칙 탐색을 통해 시퀀스 데이터에 대한 지지도, 신뢰도가 높은 페이지 순서를 찾았다.
* 부모 페이지를 기준으로 자식 페이지로 연결될 수 있게끔 내부 구성을 변화시킨다.

In [ ]:
# 지금까지 웹을 방문한 고객들의 시퀀스 데이터를 통해 신뢰도, 지지도가 높은 패턴을 발견했다.
# 그리고 이를 통해서 개별 페이지의 구성을 변화하려 한다.

In [ ]:
correlation_dict

{'페이지B': ['페이지J', '페이지I', '페이지G'],
 '페이지A': ['페이지C', '페이지E', '페이지I'],
 '페이지J': ['페이지D', '페이지C', '페이지E'],
 '메인': ['페이지H', '페이지C', '페이지B'],
 '페이지H': ['페이지J', '페이지H', '페이지E'],
 '페이지I': ['페이지A', '페이지J', '페이지F'],
 '페이지F': ['페이지H', '페이지D', '페이지A'],
 '페이지G': ['페이지G', '페이지D', '페이지E'],
 '페이지D': ['페이지E', '페이지D', '페이지C'],
 '페이지E': ['페이지I', '페이지J', '페이지D'],
 '페이지C': ['페이지F', '페이지B', '페이지H'],
 '페이지B 페이지F': ['페이지J'],
 '페이지B 페이지D': ['페이지B'],
 '페이지I 메인': ['페이지B'],
 '페이지G 페이지B': ['페이지F'],
 '페이지F 페이지J': ['페이지D'],
 '페이지J 페이지J': ['페이지C', '페이지D', '페이지F'],
 '페이지A 페이지G 페이지J': ['페이지E'],
 '페이지J 페이지I 페이지D': ['페이지F'],
 '페이지J 메인 페이지J': ['페이지F'],
 '페이지D 페이지J 페이지J': ['페이지F'],
 '페이지B 페이지E 페이지F': ['페이지G'],
 '페이지C 페이지A 페이지H': ['페이지B'],
 '페이지C 메인 페이지A': ['페이지J'],
 '페이지I 페이지H': ['페이지E'],
 '페이지A 페이지F': ['페이지C'],
 '페이지D 페이지J': ['페이지J 페이지F'],
 '페이지F 페이지B': ['페이지J'],
 '페이지I 페이지D': ['페이지F'],
 '페이지I 페이지E': ['페이지E'],
 '페이지A 페이지H': ['페이지B'],
 '페이지G 페이지J': ['페이지E'],
 '페이지J 페이지I': ['페이지D 페이지F'],
 '페이지D 페이지A': ['페이지E'],
 '페이지B 페이지

In [ ]:
correlation_dict['페이지B']

['페이지J', '페이지I', '페이지G']

> **해결 방안** 

```
correlation_dict을 참고하여 지지도와 신뢰도가 높은 상위 3개를 선별한다.

1. 이들을 부모 집합의 페이지 내에서 컨텐츠내 링크로 연결한다.
2. 개별 페이지마다 사이드바를 생성해서 이들을 링크로 연결한다.

json 파일에 담긴 페이지를 고려해서 페이지의 구성을 변경한다.
- 고객들이 자주 방문하는 페이지를 자연스럽게 연결할 수 있다.
- Internal Back Link 생성으로 인해 개별 Page의 SEO를 높일 수 있다.
```

In [ ]:
import json

with open('./association_result.json', 'w', encoding='utf-8') as file:
    json.dump(correlation_dict, file, indent='\t', ensure_ascii=False)

In [ ]:
file_path = "./association_result.json"

with open(file_path, 'r') as file:
    association_result = json.load(file)

association_result

{'페이지B': ['페이지J', '페이지I', '페이지G'],
 '페이지A': ['페이지C', '페이지E', '페이지I'],
 '페이지J': ['페이지D', '페이지C', '페이지E'],
 '메인': ['페이지H', '페이지C', '페이지B'],
 '페이지H': ['페이지J', '페이지H', '페이지E'],
 '페이지I': ['페이지A', '페이지J', '페이지F'],
 '페이지F': ['페이지H', '페이지D', '페이지A'],
 '페이지G': ['페이지G', '페이지D', '페이지E'],
 '페이지D': ['페이지E', '페이지D', '페이지C'],
 '페이지E': ['페이지I', '페이지J', '페이지D'],
 '페이지C': ['페이지F', '페이지B', '페이지H'],
 '페이지B 페이지F': ['페이지J'],
 '페이지B 페이지D': ['페이지B'],
 '페이지I 메인': ['페이지B'],
 '페이지G 페이지B': ['페이지F'],
 '페이지F 페이지J': ['페이지D'],
 '페이지J 페이지J': ['페이지C', '페이지D', '페이지F'],
 '페이지A 페이지G 페이지J': ['페이지E'],
 '페이지J 페이지I 페이지D': ['페이지F'],
 '페이지J 메인 페이지J': ['페이지F'],
 '페이지D 페이지J 페이지J': ['페이지F'],
 '페이지B 페이지E 페이지F': ['페이지G'],
 '페이지C 페이지A 페이지H': ['페이지B'],
 '페이지C 메인 페이지A': ['페이지J'],
 '페이지I 페이지H': ['페이지E'],
 '페이지A 페이지F': ['페이지C'],
 '페이지D 페이지J': ['페이지J 페이지F'],
 '페이지F 페이지B': ['페이지J'],
 '페이지I 페이지D': ['페이지F'],
 '페이지I 페이지E': ['페이지E'],
 '페이지A 페이지H': ['페이지B'],
 '페이지G 페이지J': ['페이지E'],
 '페이지J 페이지I': ['페이지D 페이지F'],
 '페이지D 페이지A': ['페이지E'],
 '페이지B 페이지